## 1 - import packages

In [1]:
import numpy as np
from pygem import FFD
from tqdm import trange

import smithers.io
import smithers.io.openfoam.openfoamhandler

## 2 - read **refCase** with ___smithers___ and transfer data to np.ndarray

In [2]:
snapshotsData = smithers.io.openfoam.OpenFoamHandler.read("simulation_data/refCase")

Snapshot acqusition in progress...
[######################################################################] 100.0 %


In [3]:
meshPoints = snapshotsData['0']['points']

## 3 - Using **PyGem** to deform the mesh. The write_points and also save to vtk file 
### 3.1 - define some functions: save_vtk funtions; ffd_save functions
### 3.2 - create **FFD** object. Set **FFD** parameters (ctr, weights), as well as NUM_SAMPLES

In [4]:
# import vtk

# def modify_and_save_vtk_mesh(new_point, ref_mesh_file, new_mesh_file):
#     """
#     Modifies the point coordinates of a VTK mesh using the provided new_point array
#     and saves the modified mesh to a new VTK file.

#     Args:
#         new_point (np.ndarray): A 1D NumPy array containing the modified point coordinates.
#         ref_mesh_file (str): Path to the reference VTK mesh file.
#         new_mesh_file (str): Path to save the new modified VTK mesh file.
#     """
#     # Read the reference mesh from the VTK file
#     reader = vtk.vtkXMLMultiBlockDataReader()
#     reader.SetFileName(ref_mesh_file)
#     reader.Update()
#     ref_mesh = reader.GetOutput()

#     # Create a new VTK mesh with the modified points
#     new_mesh = vtk.vtkPolyData()
#     new_points = vtk.vtkPoints()

#     for i in range(len(new_point)):
#         new_points.InsertNextPoint(new_point[i])

#     new_mesh.SetPoints(new_points)

#     # Write the new mesh to a separate VTK file
#     writer = vtk.vtkPolyDataWriter()
#     writer.SetFileName(new_mesh_file)
#     writer.SetInputData(new_mesh)
#     writer.Write()

In [14]:
# define a function to apply the FFD to a mesh, 
# the variable of the function are: 1- ffd object , 2- NUM_SAMPLES, 3- displacement ratio, 
#                                   4- weight matrix, 5-refMeshPoints, 6- refVTKMeshFile

def apply_ffd(ffd, NUM_SAMPLES, disp_ratio, weight_matrix, refMeshPoints, save_dir):

    # create a empty array to store the displacement data
    displacement_data = None

    for i in trange(NUM_SAMPLES):
        ffd.array_mu_x=disp_ratio*np.random.uniform(-1, 1, size=ffd.array_mu_x.shape)*weight_matrix
        ffd.array_mu_y=disp_ratio*np.random.uniform(-1, 1, size=ffd.array_mu_x.shape)*weight_matrix

        ffd.array_mu_x[:, :, 0] = ffd.array_mu_x[:, :, 1]
        ffd.array_mu_y[:, :, 0] = ffd.array_mu_y[:, :, 1]

        new_p = ffd(refMeshPoints)

        # save the new mesh points
        smithers.io.openfoam.OpenFoamHandler.write_points(new_p, f'{save_dir}/points_{i}', f'{save_dir}/points_ref')

        # save the displacement of ffd control points, including array_mu_x, array_mu_y, array_mu_z. 
        # save them into a npy file
        if i == 0:
            displacement_data = np.array([ffd.array_mu_x, ffd.array_mu_y, ffd.array_mu_z]).reshape(1, -1)
        else:
            displacement_data = np.append(displacement_data, 
                                          np.array([ffd.array_mu_x, ffd.array_mu_y, ffd.array_mu_z]).reshape(1, -1), 
                                          axis=0)

    np.save(f'{save_dir}/displacement_data.npy', displacement_data)

In [15]:
# create the FFD object, with control points 5x5x2
ffd = FFD([5, 5, 2])

# create the bounding box
ffd.box_origin = np.array([-0.01, -0.01, -0.001])
ffd.box_length = np.array([0.02, 0.02, 0.002])

# define the weight matrix. The boundary control points are fixed
weights = np.zeros(ffd.array_mu_x.shape)
weights[1:-1, 1:-1, :] = 1

# define the number of mesh samples, and the displacement ratio for the control points 
a=0.1
NUM_SAMPLES = 100

In [16]:
# apply the FFD to the mesh points

apply_ffd(ffd, NUM_SAMPLES, a, weights, meshPoints, "mesh_data")

100%|██████████| 100/100 [00:02<00:00, 39.51it/s]
